In [2]:
import random

def csv_to_array(filename):
    file = open(filename, 'r')
    file_text = file.read()
    # Get rid of the first three, which are examples
    split_text = file_text.split('|')[3:]
    i = 0
    tuple_text = []
    while i < len(split_text) - 1:
        tuple_text.append((split_text[i+1], split_text[i], int(i/2)))
        i += 2
    
    return tuple_text
    
def make_training_and_dev_data(all_data):
    random.shuffle(all_data)
    training_ratio = .9
    return all_data[:int(len(all_data) * training_ratio)], all_data[int(len(all_data) * training_ratio):]
#     return all_data[:(len(all_data) * training_ratio)], all_data[:(len(all_data) * training_ratio)] 


official_data = csv_to_array('yelp_data_official_training.csv')

for garbage in official_data:
    if garbage[1].isdigit() is False:
        print(garbage[1])
        break

training_data, dev_data = make_training_and_dev_data(official_data)

In [3]:
def reviews(data):
    reviews = []
    for i in range(len(training_data)):
        reviews.append(training_data[i][0])
    return reviews

def labels(data):
    reviews = []
    for i in range(len(training_data)):
        reviews.append(int(training_data[i][1]))
    return reviews

reviews_training = reviews(training_data)
label_training = labels(training_data)
reviews_dev = reviews(dev_data)
label_dev = labels(dev_data)

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(reviews_training)
X_train_counts.shape

/Users/RK/anaconda/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


(43200, 88826)

In [5]:
count_vect.vocabulary_.get('and')

45719

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape

(43200, 88826)

In [7]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(43200, 88826)

In [8]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, label_training)

In [9]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])

In [10]:
text_clf = text_clf.fit(reviews_training, label_training)

In [11]:
import numpy as np
docs_dev = reviews_dev
predicted = text_clf.predict(docs_dev)
np.mean(predicted == label_dev)

0.74064814814814817

In [12]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42)),])
_ = text_clf.fit(reviews_training, label_training)
predicted = text_clf.predict(docs_dev)
np.mean(predicted == label_dev) 

0.85442129629629626

In [13]:
from sklearn import metrics
print(metrics.classification_report(label_training, predicted))

             precision    recall  f1-score   support

          1       0.85      0.95      0.90     13602
          2       0.82      0.87      0.84      7158
          3       0.85      0.94      0.89     10218
          4       0.92      0.68      0.78      7012
          5       0.85      0.76      0.80      3353
          6       0.97      0.42      0.59      1857

avg / total       0.86      0.85      0.85     43200



In [14]:
metrics.confusion_matrix(label_training, predicted)

array([[12925,   125,   471,    76,     3,     2],
       [  530,  6237,   164,   217,     5,     5],
       [  403,    93,  9651,    66,     5,     0],
       [  814,  1062,   353,  4770,     8,     5],
       [   74,    20,   693,     5,  2545,    16],
       [  489,    68,    37,    42,   438,   783]])

In [15]:
import re

def test_file_to_array(filename):
    file = open(filename, 'r')
    file_text = file.read()
    # Skip the first one because it's the header starter shit
    split_text = re.split('\d+\|', file_text)[1:]
    tupled_text = []
    for i in range(len(split_text)):
        tupled_text.append((split_text[i], i))        
    return tupled_text

test_array = test_file_to_array('yelp_data_official_test_nocategories.csv')

In [64]:
test_array[1]

('My mother passed away recently at the age of 90. Dr. Cohen treated my mom for the last 3-4 years of her life. I cannot say enough good things about the way my mom was treated by Dr. Cohen. I always went with Mom for her appts. and all of our questions and concerns were quickly and appropriately addressed. My mother loved Dr. Cohen and I will forever be grateful for the loving care she gave my mom. I have a wonderful primary care physician that I have been seeing for many years and he knows all of my issues. I may try to find a way to utilize the best of both of these wonderful docs.\n',
 1)

In [81]:
test_reviews = []
test_id = []
for i in test_array:
    test_reviews.append(i[0])
    test_id.append(i[1])
    

In [82]:
test_id[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [83]:
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer()),('tfidf', TfidfTransformer()), ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, n_iter=5, random_state=42)),])
_ = text_clf.fit(reviews_training, label_training)
predicted = text_clf.predict(test_reviews)
output = []
for i in range(0,len(predicted)):
    output.append((predicted[i],int(i)))

In [86]:
output[0]

(3, 0)

In [40]:
np.argwhere(np.isnan(predicted))

array([], shape=(0, 1), dtype=int64)

In [58]:
def write_output_to_csv(output, filename):
    file = open(filename, 'w')
    file.write('Id' + ',' + 'Category' + '\n')
    for item in output:
        file.write(str(item[1]) + ',' + str(item[0]) + '\n')

write_output_to_csv(output, 'kuvinka_nicholas_kaggle_svm.csv')